In [2]:
import numpy as np
from collections import Counter

# Node Class

In [1]:
class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, *, value=None):
        self.feature = feature  # dataset column number
        self.threshold = threshold  # numeric value used to decide where to split tree at
        self.left = left        # left child
        self.right = right      # right child
        self.value=None # if leaf node or not

    def is_leaf(self):
        return self.value is not None

print('hello world!')

hello world!


# Decision Tree Class

In [ ]:
class Decision_Tree:
    def __init__(self, min_samples_split = 2, max_depth = 100, n_features=None):
        """Decision Tree constructor function."""
        self.min_samples_split=min_samples_split
        self.max_depth=max_depth
        self.n_features=n_features
        self.root=None

    def fit(self, X, y):
        """ ? """
        
        # shape gives (n_samples, n_features)
        self.n_features = X.shape[1] if not self.n_features else min(X.shape[1], self.n_features)
        
        # start building tree recursively, starting at root nodek
        self.root = self._grow_tree(X, y)

    def _grow_tree(self, X, y, depth=0):
        """ Recursive function that builds subtree beneath current node. """
        n_samples, n_feats = X.shape    # 
        n_labels = len(np.unique(y))    # get number of unique labels (usually colors in teaching demos)

        # Base Case: Check stopping criteria
        # if depth of node is past max_depth, stop splitting
        # if n_labels == 1, then it is a leaf node, stop splitting
        # if number of elts in the sample < defined min_samples number, stop splitting
        if depth >= self.max_depth or n_labels == 1 or n_samples < self.min_samples_split:
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)
        
        # Find the best split, using Information Gain & Entropy

        # Randomly pick which features to use in split, and pass them to best_split function
        feat_idxs = np.random.choice(n_feats, self.n_features, replace=False) #TODO: fix this error, feat_idxs is int, while used as iterable
        best_feature, best_thresh = self._best_split(X, y, feat_idxs)

        # Create Child nodes
        left_idxs, right_idxs = self._split(X[:, best_feature], best_thresh)
        left = self._grow_tree(X[left_idxs, :], y[left_idxs], depth + 1)
        right = self._grow_tree(X[right_idxs, :], y[right_idxs], depth + 1)

        return Node(best_feature, best_thresh, left, right)

    def _best_split(self, X, y, feat_idxs):
        """Find best possible threshold, or value to split at in all possible thresholds."""
        best_gain = -1
        split_idx, split_threshold = None, None

        for feat_idx, in feat_idxs:
            X_col = X[:, feat_idx]
            thresholds = np.unique(X_col)   # get all possible unique values to split at

            # Travese all thresholds and find threshold that gets most information gain
            for thr in thresholds:
                # Calculate information gain
                gain = self._calculate_info_gain(y, X_col, thr)
                if gain > best_gain:
                    best_gain = gain
                    split_idx = feat_idx
                    split_threshold = thr
            return split_idx, split_threshold

    def _calculate_info_gain(self, y, X_col, threshold):
        """Calculates information gain of splitting at given threshold."""
        # get parent entropy
        parent_entropy = self._calculate_entropy(y)
        # create children
        left_idxs, right_idxs = self._split(X_col, threshold)
            # no info gain if node not split into two different groups
        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return 0

        # calculate weighted avg of entropy of children
        n = len(y)  # number of samples we have in y
        n_left, n_right = len(left_idxs), len(right_idxs)
        entropy_left, entropy_right = self._calculate_entropy([left_idxs]), self._calculate_entropy([right_idxs])
        child_entropy = (n_left / n) * entropy_left + (n_right / n) * entropy_right

        # final information gain calculation
        information_gain = parent_entropy - child_entropy
        return information_gain
    
    def _split(self, X_col, spltting_threshold):
        """Splits values into two groups. 
        Less than or equal to split threshold or greater than split threshold.
        Returns indicies of 'left' and 'right' values (less than or greater than) in a tuple. """
        left_idxs = np.argwhere(X_col <= spltting_threshold).flatten()
        right_idxs = np.argwhere(X_col > spltting_threshold).flatten()
        return left_idxs, right_idxs

    def _calculate_entropy(self, y):
        """ Calculates entropy, measure of order/disorder of given data. """
        histogram = np.bincount(y)  # bincount counts number of times a value has appeared in the numpy array, y
        all_probabilities = histogram / len(y) # gets associated probabilities of all values in numpy array, y

        entropy = -1 * np.sum([p * np.log2(p) for p in all_probabilities if p > 0]) # using list comp, apply entropy formula
        return entropy


    def _most_common_label(self, y):
        """Gets most common label value of given numpy array using Counter"""
        counter = Counter(y)
        value = counter.most_common(1)[0][0]    # Most common tuple, and get first element
        return value

    def predict(self, X):
        np.array([self._traverse_tree(x, self.root) for x in X])
            
    def _traverse_tree(self, x, node: Node):
        if node.is_leaf():
            return node.value
        # find feature node is divided by, and value is less than threshold,
        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left)
        else:
            return self._traverse_tree(x, node.right)

